In [7]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC

In [2]:
data = load_iris()

In [3]:
df =pd.DataFrame(data.data , columns=data.feature_names)

In [5]:
df["target"]=data.target

In [6]:
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [85]:
X = df.drop("target",axis=1)
y= df["target"]

In [86]:
from sklearn.model_selection import train_test_split

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=42)

In [88]:
model=SVC()
model.fit(X_train,y_train)
model.score(X_test,y_test)

1.0

In [89]:
clf= GridSearchCV(SVC(gamma="scale"),{'kernel':['rbf','linear', 'poly'], 'C':[1,10,20]}
,cv=3 ,return_train_score=False)

In [90]:
clf.fit(X, y)

GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': [1, 10, 20], 'kernel': ['rbf', 'linear', 'poly']})

In [91]:
df1 =pd.DataFrame(clf.cv_results_)

In [92]:
df1

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002333,0.001393,0.002236,0.003162,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.96,0.98,0.94,0.960000,0.016330,7
1,0.002881,0.004075,0.008145,0.006794,1,linear,"{'C': 1, 'kernel': 'linear'}",1.00,1.00,0.98,0.993333,0.009428,1
2,0.007992,0.000269,0.000000,0.000000,1,poly,"{'C': 1, 'kernel': 'poly'}",0.98,0.94,0.98,0.966667,0.018856,5
3,0.005947,0.008411,0.005596,0.007119,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.98,1.00,0.94,0.973333,0.024944,3
4,0.004208,0.005951,0.005251,0.007425,10,linear,"{'C': 10, 'kernel': 'linear'}",1.00,0.94,0.98,0.973333,0.024944,3
5,0.004824,0.003415,0.003962,0.004105,10,poly,"{'C': 10, 'kernel': 'poly'}",1.00,0.92,0.96,0.960000,0.032660,7
6,0.000587,0.000830,0.005173,0.007316,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.98,0.98,0.98,0.980000,0.000000,2
7,0.010758,0.003907,0.000000,0.000000,20,linear,"{'C': 20, 'kernel': 'linear'}",1.00,0.94,0.96,0.966667,0.024944,5
8,0.000000,0.000000,0.010665,0.007575,20,poly,"{'C': 20, 'kernel': 'poly'}",1.00,0.92,0.96,0.960000,0.032660,7


In [93]:
df2 =df1[["param_C","param_kernel","mean_test_score","rank_test_score"]]

In [94]:
df2.set_index('rank_test_score',inplace=True)

In [95]:
df2.sort_index()

,param_C,param_kernel,mean_test_score
rank_test_score,,,
1,1,linear,0.993333
2,20,rbf,0.980000
3,10,rbf,0.973333
3,10,linear,0.973333
5,1,poly,0.966667
5,20,linear,0.966667
7,1,rbf,0.960000
7,10,poly,0.960000
7,20,poly,0.960000


In [96]:
clf.best_estimator_

SVC(C=1, kernel='linear')

In [97]:
clf.best_params_

{'C': 1, 'kernel': 'linear'}

In [98]:
clf.best_score_

0.9933333333333333

In [116]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import warnings 
warnings.filterwarnings("ignore")

In [117]:
model_params= {
    'svm': {
        'model':SVC(gamma='auto'),
        'params':{
             'C':[1,10,20],
             'kernel':["rbf","linear"]
                 }
           },
        'random_forest':{
            "model":RandomForestClassifier(),
            "params":{
                'n_estimators':[1,5,10]
            }
        },
        'logistic_regression':{
            'model':LogisticRegression(),
            'params':{
                "C":[1,5,10]
            }
        }
    }

In [118]:
score=[]

for model_name , mp in model_params.items():
    clf = GridSearchCV(mp["model"],mp["params"],cv=5)
    clf.fit(data.data,data.target)
    score.append({"model":model_name,
                 "best_score":clf.best_score_,
                 "best_params":clf.best_params_
                 }
                )

In [120]:
df= pd.DataFrame(score, columns=["model","best_score","best_params"])

In [121]:
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.966667,{'n_estimators': 10}
2,logistic_regression,0.980000,{'C': 10}
